# 🏔️ Sistem Rekomendasi Rute Pendakian Gunung
## Content-Based Filtering dengan SBERT dan Cosine Similarity

**Paper:** "Sistem Rekomendasi Rute Pendakian Gunung Menggunakan Content-Based Filtering Berbasis SBERT dan Cosine Similarity"

---

### Komponen Utama:
1. **Data Collection & Fusion**: Ekstraksi fitur dari GPX + deskripsi manual
2. **Preprocessing**: Regex cleaning, case folding, stopword removal (selektif)
3. **Feature Extraction**: SBERT (paraphrase-multilingual-MiniLM-L12-v2) → 384 dimensi
4. **Similarity & Ranking**: Cosine Similarity dengan Top-N ranking

In [ ]:
#@title 1️⃣ Install Dependencies
!pip install gpxpy sentence-transformers scikit-learn pandas numpy -q
print("✅ Dependencies installed!")

In [ ]:
#@title 2️⃣ Import Libraries
import os
import re
import json
import math
import numpy as np
import pandas as pd
import gpxpy
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import files
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported!")

In [ ]:
#@title 3️⃣ Text Preprocessing (Sesuai Paper)

STOPWORDS_ID = {
    'yang', 'dan', 'di', 'ke', 'dari', 'ini', 'itu', 'dengan', 'untuk', 'pada',
    'adalah', 'sebagai', 'dalam', 'juga', 'atau', 'ada', 'oleh', 'akan', 'sudah',
    'saya', 'kami', 'kita', 'mereka', 'dia', 'ia', 'anda', 'tersebut', 'dapat',
    'bisa', 'harus', 'telah', 'lalu', 'kemudian', 'serta', 'maupun', 'saat',
    'ketika', 'bila', 'kalau', 'jika', 'karena', 'agar', 'supaya', 'hingga',
    'sampai', 'antara', 'seperti', 'yaitu', 'yakni', 'bahwa', 'namun', 'tetapi'
}

PRESERVE_WORDS = {
    'tidak', 'bukan', 'jangan', 'belum', 'tanpa',
    'mudah', 'sulit', 'curam', 'landai', 'panjang', 'pendek',
    'tinggi', 'rendah', 'sejuk', 'panas', 'dingin', 'indah', 'bagus',
    'pemula', 'berpengalaman', 'santai', 'menantang', 'ekstrem'
}

def preprocess_text(text, remove_stopwords=True):
    if not text:
        return ""
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'[^\w\s\-]', ' ', text)
    text = re.sub(r'\b\d+\b', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = text.lower()
    if remove_stopwords:
        words = text.split()
        filtered_words = [w for w in words if w in PRESERVE_WORDS or w not in STOPWORDS_ID]
        text = ' '.join(filtered_words)
    return text

print("✅ Preprocessing functions defined!")

In [ ]:
#@title 4️⃣ GPX Processing Functions

def parse_gpx(gpx_path):
    with open(gpx_path, 'r', encoding='utf-8') as gpx_file:
        gpx = gpxpy.parse(gpx_file)
    points = []
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                points.append({
                    'lat': point.latitude,
                    'lon': point.longitude,
                    'ele': point.elevation if point.elevation else 0
                })
    return gpx, points

def calculate_statistics(gpx, points):
    distance_km = gpx.length_3d() / 1000.0
    elevation_gain = sum(max(0, points[i]['ele'] - points[i-1]['ele']) for i in range(1, len(points)))
    naismith_duration = (distance_km / 5) + (elevation_gain / 600)
    average_grade = (elevation_gain / (distance_km * 1000)) * 100 if distance_km > 0 else 0
    
    if average_grade < 5: difficulty = "mudah"
    elif average_grade < 10: difficulty = "sedang"
    elif average_grade < 15: difficulty = "sulit"
    else: difficulty = "sangat sulit"
    
    elevations = [p['ele'] for p in points if p['ele'] > 0]
    return {
        'distance_km': round(distance_km, 2),
        'elevation_gain_m': int(elevation_gain),
        'naismith_duration_hour': round(naismith_duration, 2),
        'average_grade_pct': round(average_grade, 2),
        'min_elevation': int(min(elevations)) if elevations else 0,
        'max_elevation': int(max(elevations)) if elevations else 0,
        'difficulty': difficulty
    }

def generate_narrative(stats, manual_description=""):
    jarak_desc = "pendek" if stats['distance_km'] < 3 else "sedang" if stats['distance_km'] < 7 else "panjang"
    waktu_desc = "singkat" if stats['naismith_duration_hour'] < 2 else "sedang" if stats['naismith_duration_hour'] < 4 else "lama"
    
    if stats['average_grade_pct'] < 5: grade_desc = "landai dan ramah pemula"
    elif stats['average_grade_pct'] < 10: grade_desc = "menantang dengan kemiringan sedang"
    elif stats['average_grade_pct'] < 15: grade_desc = "curam dan membutuhkan stamina yang baik"
    else: grade_desc = "sangat curam dan berbahaya"
    
    narrative = f"Jalur pendakian dengan jarak {jarak_desc} sekitar {stats['distance_km']} kilometer. Estimasi waktu tempuh {waktu_desc} selama {stats['naismith_duration_hour']} jam. Karakteristik jalur {grade_desc} dengan grade rata-rata {stats['average_grade_pct']}%. Total kenaikan elevasi {stats['elevation_gain_m']} meter. Tingkat kesulitan: {stats['difficulty']}."
    
    if manual_description:
        narrative = manual_description + " " + narrative
    return narrative

print("✅ GPX processing functions defined!")

In [ ]:
#@title 5️⃣ Load SBERT Model
print("⏳ Loading SBERT model...")
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
print(f"✅ Model loaded! Embedding dimension: {model.get_sentence_embedding_dimension()}")

In [ ]:
#@title 6️⃣ Core Functions

routes_database = []

def generate_embedding(text):
    return model.encode(text).tolist()

def calc_similarity(query_emb, route_emb):
    return float(cosine_similarity(
        np.array(query_emb).reshape(1, -1),
        np.array(route_emb).reshape(1, -1)
    )[0][0])

def add_route(name, gpx_path, manual_description=""):
    try:
        gpx, points = parse_gpx(gpx_path)
        if not points: return None
        stats = calculate_statistics(gpx, points)
        narrative = generate_narrative(stats, manual_description)
        embedding = generate_embedding(preprocess_text(narrative, False))
        route = {
            'id': len(routes_database) + 1,
            'name': name,
            'stats': stats,
            'narrative_text': narrative,
            'embedding': embedding
        }
        routes_database.append(route)
        print(f"✅ Added: {name} | {stats['distance_km']}km | {stats['difficulty']}")
        return route
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

def search_routes(query, top_n=5):
    if not routes_database: return []
    processed = preprocess_text(query, True)
    query_emb = generate_embedding(processed)
    results = [{'route': r, 'similarity': calc_similarity(query_emb, r['embedding'])} for r in routes_database]
    results.sort(key=lambda x: x['similarity'], reverse=True)
    return results[:top_n]

print("✅ Core functions defined!")

In [ ]:
#@title 7️⃣ Upload GPX Files
print("📁 Upload file GPX Anda:")
uploaded = files.upload()

for filename in uploaded.keys():
    if filename.endswith(('.gpx', '.xml')):
        name = filename.replace('.gpx', '').replace('.xml', '').replace('_', ' ').title()
        desc = input(f"Deskripsi untuk {name} (Enter untuk skip): ")
        add_route(name, filename, desc)

print(f"\n📚 Total: {len(routes_database)} rute")

In [ ]:
#@title 8️⃣ Demo dengan Data Sample (jika tidak upload GPX)
if not routes_database:
    samples = [
        ('Gunung Prau', 'Pemandangan sunrise, padang rumput', 3.5, 450, 'mudah'),
        ('Gunung Merbabu', 'Sabana luas, sumber air di pos 2', 12.0, 1200, 'sulit'),
        ('Gunung Sindoro', 'Jalur curam, hutan pinus', 8.0, 1400, 'sangat sulit'),
        ('Bukit Sikunir', 'Jalur pendek sunrise', 1.5, 150, 'mudah'),
        ('Gunung Sumbing', 'Trek panjang, kawah, air terjun', 10.0, 1800, 'sangat sulit'),
    ]
    for name, desc, dist, ele, diff in samples:
        stats = {
            'distance_km': dist, 'elevation_gain_m': ele,
            'naismith_duration_hour': round((dist/5) + (ele/600), 2),
            'average_grade_pct': round((ele/(dist*1000))*100, 2),
            'min_elevation': 1500, 'max_elevation': 2500 + ele, 'difficulty': diff
        }
        narrative = generate_narrative(stats, desc)
        routes_database.append({
            'id': len(routes_database) + 1, 'name': name, 'stats': stats,
            'narrative_text': narrative,
            'embedding': generate_embedding(preprocess_text(narrative, False))
        })
        print(f"✅ {name}")
    print(f"\n📚 {len(routes_database)} sample routes loaded")

In [ ]:
#@title 9️⃣ Pencarian dengan Natural Language
query = input("🔍 Masukkan preferensi (contoh: 'jalur mudah untuk pemula'): ")

if query.strip():
    results = search_routes(query, 5)
    print("\n" + "="*60)
    print("📊 HASIL REKOMENDASI")
    print("Formula: Sim(Q,D) = (Q·D) / (||Q||×||D||)")
    print("="*60)
    for i, r in enumerate(results, 1):
        route = r['route']
        print(f"\n#{i} {route['name']}")
        print(f"   📏 {route['stats']['distance_km']}km | ⛰️ {route['stats']['elevation_gain_m']}m | 🏷️ {route['stats']['difficulty']}")
        print(f"   📊 Similarity: {r['similarity']:.4f} ({r['similarity']*100:.1f}%)")
        print(f"   🤖 {route['narrative_text'][:100]}...")

In [ ]:
#@title 🔟 Test Berbagai Query
test_queries = [
    "jalur mudah untuk pemula",
    "trek menantang elevasi tinggi",
    "pendakian singkat 2-3 jam",
    "jalur landai sabana sunrise"
]

for q in test_queries:
    print(f"\n🔍 Query: '{q}'")
    for i, r in enumerate(search_routes(q, 3), 1):
        print(f"   {i}. {r['route']['name']} - {r['similarity']:.4f}")

In [ ]:
#@title 1️⃣1️⃣ Export ke CSV
df = pd.DataFrame([{
    'ID': r['id'], 'Nama': r['name'],
    'Jarak_km': r['stats']['distance_km'],
    'Elevasi_m': r['stats']['elevation_gain_m'],
    'Durasi_jam': r['stats']['naismith_duration_hour'],
    'Grade_%': r['stats']['average_grade_pct'],
    'Kesulitan': r['stats']['difficulty'],
    'Narrative': r['narrative_text']
} for r in routes_database])

df.to_csv('hiking_routes.csv', index=False)
files.download('hiking_routes.csv')
print("✅ Exported!")